In [3]:
# This class is used for running commands.
import subprocess 

# This is an example of how to run a command. You make a variable to hold the CompletedProcess object.
# completed_dir_command = subprocess.run(["dir"], shell=True, capture_output=True)

# In order to properly print it, you must decode the bytes object of the stdout into a string, or else it will look ugly.
# print(completed_dir_command.stdout.decode("UTF-8"))

# Further documentation is here:
# https://docs.python.org/3/library/subprocess.html#module-subprocess

In [ ]:
class Pokemon_Showdown_CLI_Interface:
    # Initialize a CLI instance.
    def __init__(self):
        # This must be at the start of every command.
        self.prefix = r"node pokemon-showdown-access.js "
        # This makes logging togglable
        self.logging = False
        # Helps keep an order to executed commands in logging
        self.command_number = 1
        # Secret print debug (makes things messy) (turn on if you know what you're doing)
        self.secret_print_debug = False

    # Non-CLI functions

    # Helps reduce redundant logging code
    def log(self, string):
        # If logging is enabled,
        if(self.logging):
            # Print the log with the current command number
            print(f"[{self.command_number}] {string}")
            # Increase the command number
            self.command_number += 1

    # Toggle logging
    def toggle_logging(self):
        # Swap the value
        self.logging = not self.logging
        # Log that we ran this command
        self.log("Logging enabled")

    # CLI functions

    # Start pokemon-showdown
    # Not implemented. Running showdown as a server isn't necessary right now.

    # Generate a team
    def generate_team(self):
        # Create the command
        command = self.prefix + "generate-team"
        # Run the command
        completed_command = subprocess.run(command, shell=True, capture_output=True)
        # Print the command raw.
        if(self.secret_print_debug):
            print(completed_command)
        # Collect the output by decoding the output
        team_string = completed_command.stdout.decode()
        # Log that we ran this command
        self.log("Generated team")
        # Return the team
        return team_string


In [ ]:
pokemon_showdown_cli = Pokemon_Showdown_CLI_Interface()

pokemon_showdown_cli.secret_print_debug = True
pokemon_showdown_cli.toggle_logging()
#help_statement = pokemon_showdown_cli.help()
#print(help_statement, end='')
#team_string = pokemon_showdown_cli.generate_team()
#print(team_string)



NameError: name 'Pokemon_Showdown_CLI_Interface' is not defined

In [48]:
result = subprocess.run("node pokemon-showdown-access.js", capture_output=True)

game_log = result.stdout.decode()

import re

def evaluate_position(game_log):
    # Will store the weight of various happenings.
    eval_weight = {
        "faint": 1
    }
    # This is how we will compare "winning" and "losing"
    # Kinda inspired by chess?
    # p1 is +, p2 is -
    game_evaluation = 0

    # Gets faint lines using regex
    # ^ = start of the line
    # .* = zero or more characters
    # \b = begin word border
    # \b = end word border
    # .* = zero or more characters
    # $ = end of the line
    # Combined effect? Gets the lines where pokemon
    # faint data is contained.
    faints = re.findall(r"^.*\bfaint\b.*$", game_log, re.MULTILINE)
    if len(faints) > 0:
        for faint in faints:
            if "p1" in faint:
                print(f"[-{eval_weight["faint"]}] Player 1 had a poke faint.")
                game_evaluation -= eval_weight["faint"]
            if "p2" in faint:
                print(f"[+{eval_weight["faint"]}] Player 2 had a poke faint.")
                game_evaluation += eval_weight["faint"]
    else:
        print(f"[0] No players had any pokes faint.")

    return game_evaluation

eval = evaluate_position(game_log)
print(eval)


[+1] Player 2 had a poke faint.
1
